In [1]:
#!/usr/bin/env python2
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
from mat2py import loadmat
import cPickle as pkl

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]
 
class StimInfo:
    def __init__(self):
        self.stimid = []
        self.trials = []
        self.frames = []
        self.frames_sec = []
        self.stim_on_idx = []

# source = '/nas/volume1/2photon/projects'
# experiment = 'scenes'
# session = '20171003_JW016'
# acquisition = 'FOV1'
# functional_dir = 'functional'
source = '/nas/volume1/2photon/projects'
experiment = 'gratings_phaseMod'
session = '20171009_CE059'
acquisition = 'FOV1_zoom3x'
functional_dir = 'functional'

acquisition_dir = os.path.join(source, experiment, session, acquisition)
figdir = os.path.join(acquisition_dir, 'example_figures')


In [2]:

# ================================================================================
# frame info:
# ================================================================================
#first_frame_on = 50
stim_on_sec = 2.
iti = 1. #4.
#vols_per_trial = 15
same_order = False
# =================================================================================

# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)


In [3]:
ref['trace_dir']

u'/nas/volume1/2photon/projects/gratings_phaseMod/20171009_CE059/FOV1_zoom3x/Traces'

In [4]:
# =====================================================
# Set ROI method and Trace method:
# =====================================================
curr_roi_method = ref['roi_id'][0] #'blobs_DoG'
#curr_trace_method = ref['trace_id'] #'blobs_DoG'
trace_dir = os.path.join(ref['trace_dir'], curr_roi_method)
#trace_dir = ref['trace_dir']
# =====================================================

# Create parsed-trials dir with default format:
parsed_traces_dir = os.path.join(trace_dir, 'Parsed')
if not os.path.exists(parsed_traces_dir):
    os.mkdir(parsed_traces_dir)

# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)


# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)

# Get stim-order files:
stimorder_fns = os.listdir(path_to_paradigm_files)
stimorder_fns = sorted([f for f in stimorder_fns if 'stim_order' in f or 'stimorder' in f])
print "Found %i stim-order files, and %i TIFFs." % (len(stimorder_fns), nfiles)
if len(stimorder_fns) < nfiles:
    if same_order:
        # Same stimulus order for each file (complete set)
        stimorder_fns = np.tile(stimorder_fns, [nfiles,])

for i in stimorder_fns:
    print i

# Create stimulus-dict:
#stimdict_basename = 'stimdict'

def serialize_json(instance=None, path=None):
    dt = {}
    dt.update(vars(instance))
    return dt


Found 10 stim-order files, and 10 TIFFs.
stimorder_File001.txt
stimorder_File002.txt
stimorder_File003.txt
stimorder_File004.txt
stimorder_File005.txt
stimorder_File006.txt
stimorder_File007.txt
stimorder_File008.txt
stimorder_File009.txt
stimorder_File010.txt


In [5]:
with open(os.path.join(path_to_paradigm_files, 'parsed_trials.pkl'), 'rb') as f:
    trialdict = pkl.load(f)
trialdict.keys()

['File001',
 'File003',
 'File002',
 'File005',
 'File004',
 'File007',
 'File006',
 'File009',
 'File008',
 'File010']

In [6]:
trialdict['File001']

{u'1': {'duration': 2000.0,
  'name': u'grating-ori-120-sf-0.100000',
  'stim_off_idx': 273,
  'stim_on_idx': 183,
  'stimid': 9},
 u'10': {'duration': 2000.0,
  'name': u'grating-ori-150-sf-0.500000',
  'stim_off_idx': 2695,
  'stim_on_idx': 2605,
  'stimid': 12},
 u'11': {'duration': 2000.0,
  'name': u'grating-ori-120-sf-0.500000',
  'stim_off_idx': 2964,
  'stim_on_idx': 2874,
  'stimid': 10},
 u'12': {'duration': 2000.0,
  'name': u'grating-ori-60-sf-0.100000',
  'stim_off_idx': 3233,
  'stim_on_idx': 3144,
  'stimid': 5},
 u'13': {'duration': 2000.0,
  'name': u'grating-ori-150-sf-0.100000',
  'stim_off_idx': 3502,
  'stim_on_idx': 3413,
  'stimid': 11},
 u'14': {'duration': 2000.0,
  'name': u'grating-ori-60-sf-0.100000',
  'stim_off_idx': 3771,
  'stim_on_idx': 3682,
  'stimid': 5},
 u'15': {'duration': 2000.0,
  'name': u'grating-ori-90-sf-0.100000',
  'stim_off_idx': 4041,
  'stim_on_idx': 3951,
  'stimid': 7},
 u'16': {'duration': 2000.0,
  'name': u'grating-ori-90-sf-0.1000

In [7]:
mw = True
stimdict = dict()
for fi in range(nfiles):
    currfile= "File%03d" % int(fi+1)
       
    nframes = int(simeta[currfile]['SI']['hFastZ']['numVolumes'])
    framerate = float(simeta[currfile]['SI']['hRoiManager']['scanFrameRate'])
    volumerate = float(simeta[currfile]['SI']['hRoiManager']['scanVolumeRate'])
    frames_tsecs = np.arange(0, nframes)*(1/volumerate)

    nframes_on = stim_on_sec * volumerate
    frames_iti = round(iti * volumerate) 

    # Load stim-order:
    stim_fn = stimorder_fns[fi] #'stim_order.txt'
    with open(os.path.join(path_to_paradigm_files, stim_fn)) as f:
        stimorder = f.readlines()
    curr_stimorder = [l.strip() for l in stimorder]
    unique_stims = sorted(set(curr_stimorder), key=natural_keys)
    
    if mw is False:
        nframes_off = vols_per_trial - nframes_on # HACKYYY
        first_frame_on = 50 
        for trialnum,stim in enumerate(curr_stimorder):
            #print "Stim on frame:", first_frame_on
            if not stim in stimdict.keys():
                stimdict[stim] = dict()
            if not currfile in stimdict[stim].keys():
                stimdict[stim][currfile] = StimInfo()  #StimInfo()
            framenums = list(np.arange(int(first_frame_on-frames_iti), int(first_frame_on+(vols_per_trial))))
            frametimes = [frames_tsecs[f] for f in framenums]
            stimdict[stim][currfile].trials.append(trialnum)      
            stimdict[stim][currfile].frames.append(framenums)
            stimdict[stim][currfile].frames_sec.append(frametimes)
            stimdict[stim][currfile].stim_on_idx.append(framenums.index(first_frame_on))
            first_frame_on = first_frame_on + vols_per_trial
    else:
        for trialnum,stim in enumerate(curr_stimorder):
            currtrial = str(trialnum+1)
            if not stim in stimdict.keys():
                stimdict[stim] = dict()
            if not currfile in stimdict[stim].keys():
                stimdict[stim][currfile] = StimInfo()
            first_frame_on = int(trialdict[currfile][currtrial]['stim_on_idx'])
            framenums = list(np.arange(int(first_frame_on-frames_iti), int(trialdict[currfile][currtrial]['stim_off_idx']+(frames_iti*3))))
            frametimes = [frames_tsecs[f] for f in framenums]
            
            stimdict[stim][currfile].stimid.append(trialdict[currfile][currtrial]['name'])
            stimdict[stim][currfile].trials.append(trialnum)      
            stimdict[stim][currfile].frames.append(framenums)
            stimdict[stim][currfile].frames_sec.append(frametimes)
            stimdict[stim][currfile].stim_on_idx.append(first_frame_on)


In [16]:
list(set(stimdict[stim][currfile].stimid))[0]

u'grating-ori-30-sf-0.100000'

In [131]:
for trial in trialdict['File003'].keys():
    print trial, trialdict['File003'][trial]['stim_off_idx']-trialdict['File003'][trial]['stim_on_idx']
#trialdict['File001']['9']

24 90
20 90
21 90
22 90
23 90
1 90
3 89
2 90
5 90
4 89
7 90
6 90
9 90
8 90
11 90
10 90
13 90
12 90
15 89
14 89
17 90
16 90
19 90
18 90


In [132]:
print trialdict['File001']['9']
print trialdict['File001']['18']
print trialdict['File001']['19']

{'duration': 2000.0, 'stim_on_idx': 2336, 'name': u'grating-ori-0-sf-0.100000', 'stimid': 1, 'stim_off_idx': 2426}
{'duration': 2000.0, 'stim_on_idx': 4758, 'name': u'grating-ori-120-sf-0.500000', 'stimid': 10, 'stim_off_idx': 4848}
{'duration': 2000.0, 'stim_on_idx': 5027, 'name': u'grating-ori-120-sf-0.100000', 'stimid': 9, 'stim_off_idx': 5117}


In [133]:
for stim in stimdict.keys():
    print stim, len(stimdict[stim]['File001'].frames[1])

11 270
10 270
12 270
1 270
3 270
2 270
5 269
4 269
7 270
6 270
9 270
8 269


In [134]:

# Save to PKL:
curr_stimdict_pkl = 'stimdict.pkl' #% currfile # % currslice
print curr_stimdict_pkl
with open(os.path.join(path_to_paradigm_files, curr_stimdict_pkl), 'w') as f:
    pkl.dump(stimdict, f, protocol=pkl.HIGHEST_PROTOCOL) #, f, indent=4)

# Save to JSON:
for fi in range(nfiles):
    currfile = "File%03d" % int(fi+1)
    for stim in stimdict.keys():
        stimdict[stim][currfile] = serialize_json(stimdict[stim][currfile]) 

curr_stimdict_json = 'stimdict.json' #% currfile # % currslice
print curr_stimdict_json
with open(os.path.join(path_to_paradigm_files, curr_stimdict_json), 'w') as f:
    dump(stimdict, f, indent=4)




stimdict.pkl
stimdict.json


In [103]:
for stim in stimdict.keys():
    for filename in stimdict[stim].keys():
        for t in range(len(stimdict[stim][filename].frames)):
            if len(stimdict[stim][filename].frames[t])>270:
                print stim, filename, len(stimdict[stim][filename].frames[t])


AttributeError: 'dict' object has no attribute 'frames'